In [204]:
import pandas as pd

In [205]:
train = pd.read_csv('train.csv')
train.fillna('-', inplace=True)
train

,Id,Hotel_name,Review_Title,Review_Text,Rating
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0
...,...,...,...,...,...
2346,2346,Hyatt Regency Chennai,-,Most impressive service by staff in all areas....,80.0
2347,2347,New Woodlands,Homely villa,New woodlands chennai which gave me a homely e...,71.0
2348,2348,Samudra Residency,Nice accommodation and facilities,Awesome I liked the neatness and maintenance. ...,100.0
2349,2349,The Residency Chennai,The Residency Good Centrally located Hotel,The overall experience was good. However the w...,80.0


In [206]:
test = pd.read_csv('test.csv')
test.fillna('-', inplace=True)
test

,Id,Hotel_name,Review_Title,Review_Text
0,2351,ITC Grand Chola,Mr Neeraj,On the night of my arrival from NY I had a min...
1,2352,Hotel Pandian,-,Not so great. But it is still acceptable. Bit...
2,2353,Oyo Rooms Guindy Olympia Tech Park,Nice stay for corporate people,Been a good place to stay for people who visit...
3,2354,OYO Apartments Saidapet,Average hotel,Not worth of the money we paid.worst ac.no wat...
4,2355,Ramada Chennai Egmore,A good mid range corporate hotel,"A well located hotel, with decent sized rooms ..."
...,...,...,...,...
2347,4698,Lemon Tree Chennai,Average stay,"Compared to other lemon tree stay, this was bi..."
2348,4699,Oyo Rooms T Nagar Off Pondy Bazaar,location is not good. rude behavior. staff no ...,unpleasant stay. not easy task to reach. Morni...
2349,4700,VGP Golden Beach Resort,-,Quality of service is too bad. We arrived 12.3...
2350,4701,The Park Chennai,Over rated and overpriced Hotel,I am not sure why someone wants to spend that ...


In [207]:
from sklearn.model_selection import train_test_split

X = train.drop(columns=['Rating', 'Id'])
X['text'] = X.Hotel_name + '; ' + X.Review_Title + '; ' + X.Review_Text
X.drop(columns=['Hotel_name', 'Review_Title', 'Review_Text'], inplace=True)

y = train.Rating

X

,text
0,Park Hyatt; Refuge in Chennai; Excellent room ...
1,Hilton Chennai; Hilton Chennai; Very comfortab...
2,The Royal Regency; No worth the rating shown i...
3,Rivera; Good stay; First of all nice & courteo...
4,Park Hyatt; Needs improvement; Overall ambienc...
...,...
2346,Hyatt Regency Chennai; -; Most impressive serv...
2347,New Woodlands; Homely villa; New woodlands che...
2348,Samudra Residency; Nice accommodation and faci...
2349,The Residency Chennai; The Residency Good Cent...


Данные о названии отеля сохранил, так как оценка отеля может зависить и от него самого)

In [208]:
import re

expr = r'[^A-Za-z\s]'
parser = re.compile(expr)
for i in range(len(X.text)):
    X.text.iloc[i] = re.sub(r'\s+', ' ', re.sub(expr, '', X.text.iloc[i])).lower()
X

,text
0,park hyatt refuge in chennai excellent room an...
1,hilton chennai hilton chennai very comfortable...
2,the royal regency no worth the rating shown in...
3,rivera good stay first of all nice courteous s...
4,park hyatt needs improvement overall ambience ...
...,...
2346,hyatt regency chennai most impressive service ...
2347,new woodlands homely villa new woodlands chenn...
2348,samudra residency nice accommodation and facil...
2349,the residency chennai the residency good centr...


In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [210]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

In [211]:
to_vector = TfidfVectorizer()
X_train_vec = to_vector.fit_transform(X_train.text)

In [212]:
X_test_vec = to_vector.transform(X_test.text)

In [213]:
logreg = LogisticRegression()
logreg.fit(X_train_vec, y_train)
logreg.score(X_test_vec, y_test)

0.5031847133757962

In [214]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_vec, y_train)
lr.score(X_test_vec, y_test)

0.06078294353922653

Логрег в данной задаче работает лучше.

In [215]:
from nltk import wordnet, pos_tag
from nltk import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN

def my_lemmatizer(sent):
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])

In [216]:
X_train_lm = X_train.text.apply(lambda y: my_lemmatizer(y))
X_train_lm_vec = to_vector.fit_transform(X_train_lm)

In [217]:
X_test_lm = X_test.text.apply(lambda y: my_lemmatizer(y))
X_test_lm_vec = to_vector.transform(X_test_lm)

In [218]:
logreg = LogisticRegression()
logreg.fit(X_train_lm_vec, y_train)
logreg.score(X_test_lm_vec, y_test)

0.5074309978768577

Качество изменилось не сильно.

In [219]:
from nltk.corpus import stopwords

delete = set(stopwords.words("english"))
delete

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [220]:
def remove_stopws(text):
    processed = []
    for word in text.split():
        if word not in delete:
            processed.append(word)
    return ' '.join(processed)

In [221]:
X_clean_train = X_train_lm.apply(lambda y: remove_stopws(y))
X_clean_train_vec = to_vector.fit_transform(X_clean_train)
X_clean_train

2030    lemon tree chennai nice hotel nice experience ...
725     quality inn sabari excellent location spacious...
288     metropark vadapalani waste hotel nothing say s...
1196    ambica empire awesome hotel good service aweso...
1039             malles manotaa clean room clean bathroom
                              ...                        
90      peace inn beach temples hotel best stay india ...
2252    beverly hotel superb hotel location hotel good...
55      novotel chennai sipcot nice hotel room good se...
1770    suriya international good hotel ac promise boo...
553     mgm eastwoods nice place enjoy friend weekend ...
Name: text, Length: 1880, dtype: object

In [222]:
X_clean_test = X_test_lm.apply(lambda y: remove_stopws(y))
X_clean_test_vec = to_vector.transform(X_clean_test)
X_clean_test

687     bell hotel chennai amenity provide hotel locat...
1895    raintree st mary road stay away wine careful t...
876     beverly hotel decent star hotel heart city dec...
306     hilton chennai satisfying experience good supp...
250     savera worth money slow checkin wait min check...
                              ...                        
2082    grand treat hotel close cheap n nice shopping ...
157     radisson chennai city centre india nice multi ...
1288    sai international disastar stay although book ...
1040                                 residency tower good
1843    glm meridian good stay stay comfortable cockro...
Name: text, Length: 471, dtype: object

In [223]:
logreg = LogisticRegression()
logreg.fit(X_clean_train_vec, y_train)
logreg.score(X_clean_test_vec, y_test)

0.49256900212314225

Изменение качества несильное.

Текст чистый, нормализованный. Для повышения качества можно изменить представление текста для модели, либо добавить новые фичи.

In [224]:
from sklearn.feature_extraction.text import CountVectorizer

to_vector1 = CountVectorizer()

X_clean_train_vec1 = to_vector1.fit_transform(X_clean_train)
X_clean_test_vec1 = to_vector1.transform(X_clean_test)

In [225]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_clean_train_vec1, y_train)
logreg.score(X_clean_test_vec1, y_test)

0.5583864118895966

Лучший вариант для данной задачи - CountVectorizer.

In [226]:
cool = pd.read_csv('positive-words.txt', header=None)
cool.rename(columns={0: 'words'}, inplace=True)
cool

,words
0,a+
1,abound
2,abounds
3,abundance
4,abundant
...,...
2001,youthful
2002,zeal
2003,zenith
2004,zest


In [227]:
uncool = pd.read_csv('negative-words.txt', header=None)
uncool.rename(columns={0: 'words'}, inplace=True)
uncool

,words
0,2-faced
1,2-faces
2,abnormal
3,abolish
4,abominable
...,...
4778,zaps
4779,zealot
4780,zealous
4781,zealously


In [228]:
senti = pd.read_csv('SentiWordNet.txt', delimiter='\t')
senti

,Word,Positive,Negative,PartOfSpeech,Definition
0,able,0.125,0.000,a,(usually followed by `to') having the necessar...
1,unable,0.000,0.750,a,(usually followed by `to') not having the nece...
2,dorsal,0.000,0.000,a,facing away from the axis of an organ or organ...
3,abaxial,0.000,0.000,a,facing away from the axis of an organ or organ...
4,ventral,0.000,0.000,a,nearest to or facing toward the axis of an org...
...,...,...,...,...,...
206827,fog_up,0.000,0.125,v,"get foggy; ""The windshield fogged up"""
206828,coal,0.000,0.000,v,"burn to charcoal; ""Without a drenching rain, t..."
206829,char,0.000,0.000,v,"burn to charcoal; ""Without a drenching rain, t..."
206830,haze,0.125,0.250,v,"become hazy, dull, or cloudy"


In [229]:
X_eval_senti = X_clean_train.apply(lambda y: pd.DataFrame(map(lambda wrd: senti[senti.Word == wrd].Positive.mean() - senti[senti.Word == wrd].Negative.mean(), y.split())).mean())
X_eval_senti

,0
2030,0.044514
725,0.071118
288,-0.013296
1196,0.155088
1039,0.085484
...,...
90,0.011063
2252,0.187809
55,0.194797
1770,-0.067570


In [230]:
X_eval_senti_test = X_clean_test.apply(lambda y: pd.DataFrame(map(lambda wrd: senti[senti.Word == wrd].Positive.mean() - senti[senti.Word == wrd].Negative.mean(), y.split())).mean())
X_eval_senti_test

,0
687,0.090215
1895,0.002199
876,0.138821
306,0.140050
250,0.005725
...,...
2082,0.060229
157,0.114842
1288,-0.017345
1040,0.209105


In [270]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_eval_senti, y_train)
logreg.score(X_eval_senti_test, y_test)

0.21443736730360935

Чисто по средней эмоциональной окраске слов получается довольно неплохой результат (учитывая, что все исходные данные сжались до одного числа). С другой стороны, эмоции в отзыве должны явным образом отображать мнение пользователя. Вероятно, модель испывает проблемы с отрицанием некоторых прилагательных (например, фразу "не плохо" модель пометит, как негативную, так как увидит негативно окрашеннное слово, но не распознает его отрицание), с возможным отсутствием некоторых слов в словаре, с непредсказуемостью оценок в целом (один и тот же уровень окраса, может быть получен разными наборами слов, которые предыдущие модели учитывали, а эта нет. Информация из наборов слов могла точнее указывать на возможную оценку.

Попытаюсь использовать этот парметр в модели.

Разделю выборку на 3 части. На первой обучу логрег по чистым текстам, приведённым к векторизованной форме Этой моделью сделаю предсказания для второй части выборки. К этим предсказниям добалю столбик вычисленных значений средней эмоциональной окраски слов. По этим двум столбикам обучу новый логрег. Третью часть использую для вычисления качества полученной модели. Для этого сделаю предсказания на ней логрегом по чистым текстам, затем вычислю среднюю окраску слов в отзывах и применю к этим столбикам обученный ранее логрег.

In [341]:
X_for_pure_texts, X_to_train, y_for_pure_texts, y_to_train, _, X_eval_senti_tr = train_test_split(X_clean_train, y_train, X_eval_senti, test_size=0.2)

In [342]:
to_vector1 = CountVectorizer()

X_pure_train_vec1 = to_vector1.fit_transform(X_for_pure_texts)
X_pure_pred_vec1 = to_vector1.transform(X_to_train)

In [343]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_pure_train_vec1, y_for_pure_texts)
pred_y = logreg.predict(X_pure_pred_vec1)

new_X = pd.DataFrame(pred_y)
new_X['senti'] = X_eval_senti_tr.values

In [344]:
logreg1 = LogisticRegression(max_iter=1000)
logreg1.fit(new_X, y_to_train)

X_for_score = pd.DataFrame(logreg.predict(to_vector1.transform(X_clean_test)))
X_for_score['senti'] = X_eval_senti_test.values
logreg1.score(X_for_score, y_test)

0.3503184713375796

Сработало довольно плохо(

Но лучше, чем на чистых текстах)

Полагаю, результат связан с тем, что слмшком мало данных для обучения получает, либо первый логрег по отзывам, либо второй итоговый (в зависимости от выбранного отношения размера выборок). В случае недостаточного обучения первого логрега на втором предсказания неточны, на тесте предсказания неточны, от чего в целом модель неточна. В случае недообучения модели, объединяющей предсказания от первого логрега со средней окрашенностью слов в тексте, точный первый логрег теряется, так как итоговая модель выдает на основании этих точных данных много ошибок.

Вероятно, таким способом в целом нельзя сильно повысить качество модели, поскольку в какой-то мере данные о положительном или отрицательном окрасе содержатся в старой модели.

Итого, получилось качество в 56% для оценки от 1 до 100 на основании текстов отзывов. Полагаю, точность довольно неплоха для такой задачи.

In [352]:
X = test.drop(columns=['Id'])
X['text'] = X.Hotel_name + '; ' + X.Review_Title + '; ' + X.Review_Text
X.drop(columns=['Hotel_name', 'Review_Title', 'Review_Text'], inplace=True)

expr = r'[^A-Za-z\s]'
parser = re.compile(expr)
for i in range(len(X.text)):
    X.text.iloc[i] = re.sub(r'\s+', ' ', re.sub(expr, '', X.text.iloc[i])).lower()

X = X.text.apply(lambda y: my_lemmatizer(y))
X = X.apply(lambda y: remove_stopws(y))
X

0       itc grand chola mr neeraj night arrival ny min...
1       hotel pandian great still acceptable bit high ...
2       oyo room guindy olympia tech park nice stay co...
3       oyo apartment saidapet average hotel worth mon...
4       ramada chennai egmore good mid range corporate...
                              ...                        
2347    lemon tree chennai average stay compare lemon ...
2348    oyo room nagar pondy bazaar location good rude...
2349    vgp golden beach resort quality service bad ar...
2350    park chennai rat overpriced hotel sure someone...
2351    peace inn nice beach hotel visit chennai summe...
Name: text, Length: 2352, dtype: object

In [366]:
X_clean_train_vec1 = to_vector1.fit_transform(X_clean_train)
X_vec1 = to_vector1.transform(X)

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_clean_train_vec1, y_train)
y_pred = logreg.predict(X_vec1)

In [367]:
y_pred

array([100.,  60.,  71., ...,  40.,  40.,  80.])